In [15]:
#libs usadas no code

import pandas as pd
import numpy as np

from sklearn import metrics
from functools import reduce
from sklearn.metrics import roc_auc_score
from sklearn.utils import resample
from sklearn.model_selection import train_test_split  # Import train_test_split function
from datetime import timedelta
from datetime import date
from sklearn.metrics import confusion_matrix

from imblearn.over_sampling import SMOTE
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler


In [16]:
#transaformacap da coluna date em datetime


time_col = ['date']



data = pd.read_csv('C:/Users/fgeba/Downloads/SCORE/soccer-spi/tecnicos_br_modelagem.csv', sep=";", parse_dates=time_col)



In [17]:
data.dtypes

season                  int64
date           datetime64[ns]
league_id               int64
league                 object
team1                  object
team2                  object
proj_score1           float64
proj_score2           float64
score1                  int64
score2                  int64
tecnico                object
saiu                    int64
home                    int64
dtype: object

In [18]:
#criacao de duas variaves utilizadas no codigo 


data['ponto']=np.where(data['score1']>data['score2'],3,np.where(data['score1']==data['score2'],1,0))
data['ponto']=np.where(data['home']==1,data['ponto'],np.where(data['score1']>data['score2'],0,np.where(data['score1']==data['score2'],1,3)))                       

data['saldo_gol']=np.where(data['home']==1,data['score1']-data['score2'],data['score1']-data['score2'])


In [19]:
#criacao do lag das variaveis
data['ponto_l1'] = (data.sort_values(by=['date'], ascending=True).groupby(['tecnico'])['ponto'].shift(1))
data['ponto_l2'] = (data.sort_values(by=['date'], ascending=True).groupby(['tecnico'])['ponto'].shift(2))
data['ponto_l3'] = (data.sort_values(by=['date'], ascending=True).groupby(['tecnico'])['ponto'].shift(3))
data['ponto_l4'] = (data.sort_values(by=['date'], ascending=True).groupby(['tecnico'])['ponto'].shift(4))
data['ponto_l5'] = (data.sort_values(by=['date'], ascending=True).groupby(['tecnico'])['ponto'].shift(5))

data['saldo_gol_l1'] = (data.sort_values(by=['date'], ascending=True).groupby(['tecnico'])['saldo_gol'].shift(1))
data['saldo_gol_l2'] = (data.sort_values(by=['date'], ascending=True).groupby(['tecnico'])['saldo_gol'].shift(2))
data['saldo_gol_l3'] = (data.sort_values(by=['date'], ascending=True).groupby(['tecnico'])['saldo_gol'].shift(3))
data['saldo_gol_l4'] = (data.sort_values(by=['date'], ascending=True).groupby(['tecnico'])['saldo_gol'].shift(4))
data['saldo_gol_l5'] = (data.sort_values(by=['date'], ascending=True).groupby(['tecnico'])['saldo_gol'].shift(5))


In [20]:
#funcoes que utilizei a seguir de maximo e soma 
def soma5(v1, v2, v3,v4,v5):
    y = np.array([v1, v2, v3,v4,v5])
    return np.nansum(y)

def max5(v1, v2, v3,v4,v5):
    y = np.array([v1, v2, v3,v4,v5])
    return np.nanmax(y)




In [21]:
data['pontos_5jogos'] = data.apply(lambda row: soma5(row['ponto_l1'], row['ponto_l2'],row['ponto_l3'],row['ponto_l4'],row['ponto_l5']), axis=1)
data['pontos_4jogos'] = data.apply(lambda row: soma5(row['ponto_l1'], row['ponto_l2'],row['ponto_l3'],row['ponto_l4'],np.nan), axis=1)
data['pontos_3jogos'] = data.apply(lambda row: soma5(row['ponto_l1'], row['ponto_l2'],row['ponto_l3'],np.nan,np.nan), axis=1)
data['pontos_2jogos'] = data.apply(lambda row: soma5(row['ponto_l1'], row['ponto_l2'],np.nan,np.nan,np.nan), axis=1)
data['pontos_1jogos'] = data.apply(lambda row: soma5(row['ponto_l1'], np.nan,np.nan,np.nan,np.nan), axis=1)

data['max_saldo_5games'] = data.apply(lambda row: max5(row['saldo_gol_l1'], row['saldo_gol_l2'],row['saldo_gol_l3'],row['saldo_gol_l4'],row['saldo_gol_l5']), axis=1)
data['max_saldo_4games'] = data.apply(lambda row: max5(row['saldo_gol_l1'], row['saldo_gol_l2'],row['saldo_gol_l3'],row['saldo_gol_l4'],np.nan), axis=1)
data['max_saldo_3games'] = data.apply(lambda row: max5(row['saldo_gol_l1'], row['saldo_gol_l2'],row['saldo_gol_l3'],np.nan,np.nan), axis=1)
data['max_saldo_2games'] = data.apply(lambda row: max5(row['saldo_gol_l1'], row['saldo_gol_l2'],np.nan,np.nan,np.nan), axis=1)
data['max_saldo_1games'] = data.apply(lambda row: max5(row['saldo_gol_l1'], np.nan,np.nan,np.nan,np.nan), axis=1)






C:\Users\fgeba\anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: All-NaN slice encountered
  


In [22]:
#base de validacao 

data=data[data['ponto_l1']>=0]

d3_ott=data[data['date']>='2020-12-01']

data=data[data['date']<'2020-12-01']

In [41]:
from sklearn.model_selection import train_test_split
feature_cols = [
    'pontos_5jogos',
    'pontos_4jogos',
    'pontos_3jogos',
    'pontos_2jogos',
    'pontos_1jogos',


]

data=data[data['ponto_l1']>=0]
data['saiu'] = data['saiu'].astype(int)
    
X=data[feature_cols]

y=data['saiu']

In [42]:

oversample = SMOTE()
X, y = oversample.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 


In [43]:

clf2=RandomForestClassifier(n_estimators=10, random_state=0,max_depth=4, oob_score  =  True )

clf2.fit(X_train,y_train)

y_pred=clf2.predict(X_test)

C:\Users\fgeba\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:540: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\fgeba\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


In [44]:

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))


print('R^2 Training Score: {:.2f} \nOOB Score: {:.2f} \nR^2 Validation Score: {:.2f}'.format(clf2.score(X_train, y_train), 
                                                                                             clf2.oob_score_,
                                                                                             clf2.score(X_test, y_test)))

feature_importances = pd.DataFrame(clf2.feature_importances_, index = X.columns,  columns=['importance']).sort_values('importance',  ascending=False)
feature_importances[0:21]

[[58 19]
 [ 8 70]]
              precision    recall  f1-score   support

           0       0.88      0.75      0.81        77
           1       0.79      0.90      0.84        78

    accuracy                           0.83       155
   macro avg       0.83      0.83      0.82       155
weighted avg       0.83      0.83      0.82       155

0.8258064516129032
R^2 Training Score: 0.87 
OOB Score: 0.84 
R^2 Validation Score: 0.83


,importance
pontos_4jogos,0.358291
pontos_1jogos,0.312106
pontos_3jogos,0.144057
pontos_5jogos,0.098879
pontos_2jogos,0.086667


In [46]:
    
Xc=data[feature_cols]

yc=data['saiu']

In [47]:

y_pred=clf2.predict_proba(Xc)
df = pd.DataFrame(y_pred,columns=['good','bad'])
df2=df.reset_index(drop=True).merge(yc.reset_index(drop=True), left_index=True, right_index=True)
df2.to_csv("C:/Users/fgeba/Downloads/saida.csv")



In [48]:
    
Xc=d3_ott[feature_cols]

yc=d3_ott['saiu']

In [49]:
d3_ott.to_csv("C:/Users/fgeba/Downloads/saida.csv")

In [50]:

y_pred=clf2.predict_proba(Xc)
df = pd.DataFrame(y_pred,columns=['good','bad'])
df2=df.reset_index(drop=True).merge(d3_ott.reset_index(drop=True), left_index=True, right_index=True)
df2.to_csv("C:/Users/fgeba/Downloads/saida2.csv")